In [827]:
import pandas as pd
import numpy as np

In [870]:
data=pd.read_csv('planecrash.csv')

In [824]:
set(data['time'])


{nan,
 '21:20',
 '11:04',
 '20:08',
 '09:49',
 '08:23',
 'c 03:40',
 '04:10',
 '08:30',
 '16:56',
 '11:54',
 '1450',
 '22:54',
 '20:27',
 '02:32',
 '06:22',
 'c 16:35',
 '04:15Z',
 '12;00',
 '10:19',
 '23:26',
 '22:15',
 '0920',
 '01:35',
 '10:44',
 '09:33',
 '12:56',
 '16:52',
 'c 17:40',
 '18:04',
 '19:55',
 '19:36',
 '21:17',
 'c 17:00',
 '19:43',
 '17:38',
 '18:30',
 '13:00Z',
 '23:45',
 '06:50',
 '05:40',
 '13:49',
 'c 12:55',
 '14:32',
 '15:09',
 '15:29',
 '14;10',
 '1101',
 '15:02',
 '09:50',
 '13:59',
 '01:55',
 '18:06',
 '23:36',
 '19:27',
 '23:12',
 '19:31',
 '17:50',
 '0019',
 '11:16',
 '10:30',
 '15:26',
 'c 07:00',
 'c 17:35',
 '07:00Z',
 '22:17',
 '05:00Z',
 '08:10',
 '06:32',
 '22:24',
 '21:50',
 '10:49',
 '20:28',
 '1015',
 '17:58',
 '08:09',
 '13:45',
 '114:20',
 '20:16',
 '13:28',
 '03:59',
 '13:06',
 '23:21',
 '22:32',
 '10:31',
 'c 01:30',
 '04:12',
 '05:00',
 '14:48',
 '16:13',
 '19:46',
 '17:19',
 '12:48',
 '12:52',
 'c 13:50',
 '03:41',
 '13:08',
 '17:15',
 '0415

In [838]:
# Hay valores en time que para que sean considerados hora necesitan separaciòn
data.at[64,'time']

'1140'

In [871]:
for i in range(len(data['time'])):
    if len(data.at[i,'time'])==4:
         da=data.at[i,'time']
         da1=da[:2]+':'+da[2:]
         data.at[i,'time']=da1
 

data.at[64,'time']


'11:40'

In [872]:
# Hay datos con caracteres extras

data.at[4,'time']

'c 18:30'

In [873]:
import re
data['time'] = data.time.str.lower().str.lstrip('c').str.lstrip('d').str.rstrip('z').str.lstrip(':').str.strip()
funcion = lambda x: re.sub(r'[::;\"]', ':', x)
funcion2 = lambda x: re.sub(r'::', ':' , x)
data['time'] = data.time.apply(funcion)
data['time']=data.time.apply(funcion2)

data.at[4,'time']


'18:30'

In [841]:
data.at[759,'time']

'1:30'

In [874]:
data = data.replace('?', np.nan)
null_cols = data.isnull().sum()
drop_cols = list(null_cols[null_cols > 3000].index)
print(drop_cols)
data=data.drop(drop_cols,axis=1)

['flight_no']


In [875]:
data=data.drop('cn_ln',axis=1)

In [876]:
data = data.replace('?', np.nan)
data=data.transpose()
null_cols = data.isnull().sum()
drop_cols = list(null_cols[null_cols > 7].index)
data=data.drop(drop_cols,axis=1)


In [877]:
data=data.transpose()
data.head()

,date,time,location,operator,route,ac_type,registration,aboard,fatalities,ground,summary
0,"September 17, 1908",17:18,"Fort Myer, Virginia",Military - U.S. Army,Demonstration,Wright Flyer III,NaN,2 (passengers:1 crew:1),1 (passengers:1 crew:0),0,"During a demonstration flight, a U.S. Army fly..."
1,"September 07, 1909",NaN,"Juvisy-sur-Orge, France",NaN,Air show,Wright Byplane,SC1,1 (passengers:0 crew:1),1 (passengers:0 crew:0),0,Eugene Lefebvre was the first pilot to ever be...
2,"July 12, 1912",06:30,"Atlantic City, New Jersey",Military - U.S. Navy,Test flight,Dirigible,NaN,5 (passengers:0 crew:5),5 (passengers:0 crew:5),0,First U.S. dirigible Akron exploded just offsh...
3,"August 06, 1913",NaN,"Victoria, British Columbia, Canada",Private,NaN,Curtiss seaplane,NaN,1 (passengers:0 crew:1),1 (passengers:0 crew:1),0,The first fatal airplane accident in Canada oc...
4,"September 09, 1913",18:30,Over the North Sea,Military - German Navy,NaN,Zeppelin L-1 (airship),NaN,20 (passengers:? crew:?),14 (passengers:? crew:?),0,The airship flew into a thunderstorm and encou...


In [878]:
data['aboard']=data_2['aboard'].str.replace('\xa0','')
set(data['aboard'])

data['aboard']=data_2['fatalities'].str.replace('\xa0','')
set(data['aboard'])


{'0  (passengers:0 crew:0)',
 '0  (passengers:0 crew:2)',
 '0  (passengers:0 crew:4)',
 '0  (passengers:1 crew:1)',
 '0  (passengers:? crew:?)',
 '1  (passengers:0 crew:0)',
 '1  (passengers:0 crew:1)',
 '1  (passengers:1 crew:0)',
 '1  (passengers:1 crew:1)',
 '1  (passengers:1 crew:?)',
 '1  (passengers:5 crew:1)',
 '1  (passengers:? crew:?)',
 '10  (passengers:0 crew:10)',
 '10  (passengers:1 crew:9)',
 '10  (passengers:10 crew:0)',
 '10  (passengers:2 crew:8)',
 '10  (passengers:3 crew:7)',
 '10  (passengers:4 crew:6)',
 '10  (passengers:5 crew:5)',
 '10  (passengers:6 crew:4)',
 '10  (passengers:7 crew:3)',
 '10  (passengers:8 crew:2)',
 '10  (passengers:9 crew:1)',
 '10  (passengers:? crew:?)',
 '100  (passengers:93 crew:7)',
 '100  (passengers:? crew:?)',
 '101  (passengers:95 crew:6)',
 '102  (passengers:91 crew:11)',
 '102  (passengers:93 crew:9)',
 '102  (passengers:96 crew:6)',
 '102  (passengers:97 crew:5)',
 '103  (passengers:92 crew:11)',
 '103  (passengers:96 crew:7)',
 

In [879]:
re.split("(\s+)",data.at[2,'aboard'])[:1]

['5']

In [880]:
data['Total_Passengers']=0
data['Total_Fatalities']=0

In [881]:
def split(x):
    return pd.Series(re.split("(\s+)",x)[:1])

data['Total_Passengers'] = data['aboard'].apply(split)
data['Total_Fatalities'] = data['aboard'].apply(split)


In [ ]:
data.head()


In [882]:
def split_2(x):
    return pd.Series(re.split(':',x)[2:])


data['Total_Crew'] = data['aboard'].apply(split_2)
data['Total_Fatalities_Crew'] = data['fatalities'].apply(split_2)

data['Total_Crew']=data['Total_Crew'].str.replace(')','')
data['Total_Fatalities_Crew']=data['Total_Fatalities_Crew'].str.replace(')','')


data.head()

,date,time,location,operator,route,ac_type,registration,aboard,fatalities,ground,summary,Total_Passengers,Total_Fatalities,Total_Crew,Total_Fatalities_Crew
0,"September 17, 1908",17:18,"Fort Myer, Virginia",Military - U.S. Army,Demonstration,Wright Flyer III,NaN,1 (passengers:1 crew:0),1 (passengers:1 crew:0),0,"During a demonstration flight, a U.S. Army fly...",1,1,0,0
1,"September 07, 1909",NaN,"Juvisy-sur-Orge, France",NaN,Air show,Wright Byplane,SC1,1 (passengers:0 crew:0),1 (passengers:0 crew:0),0,Eugene Lefebvre was the first pilot to ever be...,1,1,0,0
2,"July 12, 1912",06:30,"Atlantic City, New Jersey",Military - U.S. Navy,Test flight,Dirigible,NaN,5 (passengers:0 crew:5),5 (passengers:0 crew:5),0,First U.S. dirigible Akron exploded just offsh...,5,5,5,5
3,"August 06, 1913",NaN,"Victoria, British Columbia, Canada",Private,NaN,Curtiss seaplane,NaN,1 (passengers:0 crew:1),1 (passengers:0 crew:1),0,The first fatal airplane accident in Canada oc...,1,1,1,1
4,"September 09, 1913",18:30,Over the North Sea,Military - German Navy,NaN,Zeppelin L-1 (airship),NaN,14 (passengers:? crew:?),14 (passengers:? crew:?),0,The airship flew into a thunderstorm and encou...,14,14,?,?


In [883]:
data['date'] = pd.to_datetime(data['date'])
lista=['aboard','fatalities']
data=data.drop(lista,axis=1)    


In [ ]:
#data['registration']= data['registration'].fillna(0)

In [884]:
data = data.replace('?', np.nan)

data['route']=data['route'].str.replace('?','')

In [885]:
data.dropna(subset=['Total_Crew','Total_Fatalities_Crew'],inplace=True)

data['Total_Crew']=data['Total_Crew'].astype(int)
data['Total_Passengers']=data['Total_Passengers'].astype(int)
data['Total_Fatalities']=data['Total_Fatalities'].astype(int)
data['Total_Fatalities_Crew']=data['Total_Fatalities_Crew'].astype(int)


#data['Total_Passengers_Not_Crew']=0
data['Total_Passengers_Not_Crew']=data['Total_Passengers']-data['Total_Crew']
data['Total_Fatalities_Not_Crew']=data['Total_Fatalities']-data['Total_Fatalities_Crew']

In [893]:
data.dtypes

date                         datetime64[ns]
time                                 object
location                             object
operator                             object
Type                                 object
ac_type                              object
registration                         object
ground                               object
summary                              object
Total_Passengers                      int32
Total_Fatalities                      int32
Total_Crew                            int32
Total_Fatalities_Crew                 int32
Total_Passengers_Not_Crew             int32
Total_Fatalities_Not_Crew             int32
Route                                object
dtype: object

In [887]:
data['route'].value_counts()

Training                         92
Sightseeing                      42
Test flight                      19
Test                              7
Sao Paulo - Rio de Janeiro        7
Paris - London                    5
Rio de Janeiro - Sao Paulo        4
Huambo - Luanda                   4
Bogota - Barranquilla             4
Saigon - Paris                    4
Barranquilla - Bogota             4
Paris - Croydon                   3
Guayaquil - Quito                 3
Istanbul - Ankara                 3
Strasbourg - Paris                3
Quito - Cuenca                    3
Villavicencio - Mitu              3
Skydiving                         3
Guayaquil - Cuenca                3
Berlin - Munich                   3
Burbank - Oakland                 3
Davao - Manila                    3
London - Paris                    3
Cleveland - Chicago               3
Krasnoyarsk - Irkutsk             3
Sao Paulo - Porto Alegre          3
Rome - Cairo                      3
Lagos - Abuja               

In [888]:
data.loc[(data['route']=='Test'),'route']='Test flight'

In [889]:
data['route2']=data['route']
data.loc[(data['route']!='Test flight')&(data['route']!= 'Sightseeing')&(data['route']!='Air show')&(data['route']!='Training')&(data['route']!='Demonstration')
         ,'route']='Commercial'


In [ ]:
#def split_3(x):
 #   return pd.Series(re.split(':',x)[2:])



#data['route2'] = data['route2'].apply(split_2)

In [890]:
data.dropna(subset=['route','route2'],inplace=True)

In [891]:
data=data.rename(columns={'route':'Type'})
data=data.rename(columns={'route2':'Route'})

In [895]:
data.to_csv('plane_crash.csv', index=False)
data.head()

,date,time,location,operator,Type,Route,Total_Passengers,Total_Fatalities,Total_Passengers_Not_Crew,Total_Fatalities_Not_Crew,Total_Crew,Total_Fatalities_Crew,ac_type,ground,registration,summary
0,1908-09-17,17:18,"Fort Myer, Virginia",Military - U.S. Army,Demonstration,Demonstration,1,1,1,1,0,0,Wright Flyer III,0,NaN,"During a demonstration flight, a U.S. Army fly..."
1,1909-09-07,NaN,"Juvisy-sur-Orge, France",NaN,Air show,Air show,1,1,1,1,0,0,Wright Byplane,0,SC1,Eugene Lefebvre was the first pilot to ever be...
2,1912-07-12,06:30,"Atlantic City, New Jersey",Military - U.S. Navy,Test flight,Test flight,5,5,0,0,5,5,Dirigible,0,NaN,First U.S. dirigible Akron exploded just offsh...
26,1919-08-02,NaN,"Verona, Italy",Caproni Company,Commercial,Venice Taliedo,14,14,12,12,2,2,Caproni Ca.48,0,NaN,As the plane was passing over Verona the wings...
31,1919-12-11,NaN,"Catherham, Surrey, UK",Aircraft Travel Transport,Commercial,Paris - Hounslow,1,1,0,0,1,1,de Havilland DH-4,0,G-EAHF,Crashed in a field while attemptting to land.


In [894]:

neworder = ['date','time','location','operator','Type','Route','Total_Passengers','Total_Fatalities','Total_Passengers_Not_Crew'
           ,'Total_Fatalities_Not_Crew','Total_Crew','Total_Fatalities_Crew','ac_type','ground','registration','summary']
data=data.reindex(columns=neworder)
